In [9]:
import numpy as np
from scipy.stats import norm

# Sigmoid function
def sigmoid(x, a, b):
    return 1 / (1 + np.exp(-(x - a) / b))

# Prior distribution (Gaussian)
def prior_distribution(x, mean, std):
    return norm.pdf(x, mean, std)

def update_posterior(prior, contrast, response, a_range, b):
    likelihood = sigmoid(contrast, a_range, b) if response == 1 else 1 - sigmoid(contrast, a_range, b)
    posterior = prior * likelihood
    posterior /= np.sum(posterior)  # Normalize
    return posterior

In [10]:
class ZESTIterator:
    def __init__(self, mean, std, b, a_range):
        self.a_range = a_range
        self.prior = prior_distribution(a_range, mean, std)
        self.b = b

    def __call__(self, x, r):
        for contrast, response in zip(x, r):
            self.prior = update_posterior(self.prior, contrast, response, self.a_range, self.b)
        
        # Estimate threshold as the mean of the posterior distribution
        threshold_est = np.sum(self.a_range * self.prior)
        
        # Estimate error as the standard deviation of the posterior distribution
        err = np.sqrt(np.sum((self.a_range - threshold_est) ** 2 * self.prior))
        
        # Suggest next contrast level (e.g., mean of the posterior distribution)
        x_next = self.a_range[np.argmax(self.prior)]
        
        return threshold_est, err, x_next
    
    

In [11]:
# Example usage
mean = 0.5  # Initial guess for the threshold
std = 0.1   # Standard deviation of the prior
b = 0.1     # Slope of the sigmoid function
a_range = np.linspace(-1, 1, 100)  # Range of possible threshold values

zest = ZESTIterator(mean, std, b, a_range)

# Simulate some test data
x = [0, 0.2, 0.4, 0.6]
r = [1, 1, 0, 0]

threshold_est, err, x_next = zest(x, r)
print(f"Estimated contrast threshold: {threshold_est}")
print(f"Estimated error: {err}")
print(f"Suggested next contrast level: {x_next}")

Estimated contrast threshold: 0.45011697551846686
Estimated error: 0.07545802045764448
Suggested next contrast level: 0.4545454545454546
